In [17]:
import pandas as pd
import datetime as dt

In [11]:
import os
print(os.getcwd())
os.listdir("/kaggle/input")

/kaggle/working


[]

In [13]:
# Loading the dataset
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
print(df.shape)
df.head()

(8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [15]:
# Understanding the Data
print("Shape of the dataset (R x C):", df.shape) 
print("\nColumns in the dataset:\n", df.columns.tolist())
print("\nData types:\n", df.dtypes) 
print("\nMissing values per column:\n", df.isnull().sum())
print("\nNumber of duplicate rows:", df.duplicated().sum())

Shape of the dataset (R x C): (8807, 12)

Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

Data types:
 show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

Missing values per column:
 show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

Number of duplicate rows: 0


In [20]:
# Feature extraction and Structuring
# Convert 'date_added' to datetime
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

# Split duration into value and unit 
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')
df['duration_value'] = pd.to_numeric(df['duration_value'])

In [21]:
# Step 4
# 4. Cleaning (Handling Missing and Duplicate Data) # 

# Drop duplicate rows df = df.drop_duplicates()
# Drop 'description' column (not needed) 
df = df.drop(columns=['description'])
# Handle missing Director values
df['director'] = df['director'].fillna('Not Given')
# Handle missing Cast values
df['cast'] = df['cast'].fillna('Not Given')
# Handle missing Country values
df['country'] = df['country'].fillna('Not Given')
# Drop rows with critical nulls (date_added, rating, duration) 
df = df.dropna(subset=['date_added', 'rating', 'duration'])

In [22]:
# Step 5
# 5. Transformation (Filtering, Sorting, Grouping)
# ------------------------------- 
# Example: Filter only Movies
movies = df[df['type'] == 'Movie']
# Example: Sort by most recent release year
recent_releases = df.sort_values(by='release_year', ascending=False).head(10)
# Example: Group by country to count number of titles
titles_by_country = df.groupby('country')['show_id'].count().sort_values(ascending=False).head(10)
print("\nTop 10 Countries by Number of Titles:\n", titles_by_country)


Top 10 Countries by Number of Titles:
 country
United States     2775
India              971
Not Given          826
United Kingdom     403
Japan              240
South Korea        195
Canada             173
Spain              141
France             122
Mexico             110
Name: show_id, dtype: int64


In [23]:
# Step 6: Error Checking & Validation

# Check if any date_added year is before release year 
inconsistencies = df[df['date_added'].dt.year < df['release_year']]
print("\nRecords where date_added < release_year:\n", inconsistencies)
# Ensure data types
print("\nData types after cleaning:\n", df.dtypes)
# Sample 5 random rows for manual inspection
print("\nSample records:\n", df.sample(5))
# Reset index for clean output
df = df.reset_index(drop=True)


Records where date_added < release_year:
      show_id     type                       title               director  \
1551   s1552  TV Show                       Hilda              Not Given   
1696   s1697  TV Show                Polly Pocket              Not Given   
2920   s2921  TV Show               Love Is Blind              Not Given   
3168   s3169  TV Show                Fuller House              Not Given   
3287   s3288  TV Show          Maradona in Mexico              Not Given   
3369   s3370  TV Show             BoJack Horseman              Not Given   
3433   s3434  TV Show            The Hook Up Plan              Not Given   
4844   s4845  TV Show   Unbreakable Kimmy Schmidt              Not Given   
4845   s4846  TV Show        Arrested Development              Not Given   
5394   s5395    Movie  Hans Teeuwen: Real Rancour  Doesjka van Hoogdalem   
5658   s5659  TV Show                      Sense8              Not Given   
5677   s5678  TV Show                 Tokyo T

In [24]:
# Step 7 Publish (Save Cleaned Dataset) 
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False) 
print("\nCleaned dataset exported successfully!")


Cleaned dataset exported successfully!
